In [1]:
import pandas as pd
import wrangle
import kaggle

1. pip install kaggle
2. Log-in to Kaggle (or sign up)
3. Navigate to your Account page (click top-right profile picture)

In [2]:
# Load the data
df = wrangle.wrangle_hotel()

Using cached csv file...


1. pip install kaggle

2. Log-in to Kaggle (or sign up)

3. Navigate to your Account page (click top-right profile picture)

4. API section on the Kaggle Account page.

5. Scroll down to the API section and click Create New API Token

6. Save kaggle.json to the file path displayed in the OSError message given when attempting to import kaggle

In [3]:
df.head()

,additional_number_of_scoring,average_score,hotel_name,reviewer_nationality,negative_review,review_total_negative_word_counts,total_number_of_reviews,positive_review,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given,...,positive_clean_review,negative_clean_review,positive_stem,negative_stem,positive_lemma,negative_lemma,pos_sentiment_score,neg_sentiment_score,pos_lem_sentiment_score,neg_lem_sentiment_score
0,1485,7.5,Hilton London Metropole,United Kingdom,First day the swimming pool was not opened th...,54,6977,The hotel was clean and accessible,8,4,...,hotel clean accessible,first day swimming pool opened therefore could...,hotel clean access,first day swim pool open therefor could access...,hotel clean accessible,first day swimming pool opened therefore could...,0.4019,-0.1027,0.4019,-0.1027
1,125,8.6,Hotel Mademoiselle,United Kingdom,NaN,0,853,very good location breakfast was great,7,10,...,good location breakfast great,NaN,good locat breakfast great,NaN,good location breakfast great,NaN,0.7906,0.0000,0.7906,0.0000
2,130,8.2,Vienna Marriott Hotel,Qatar,Nothing,2,939,It was very nice excellent hotel,7,4,...,nice excellent hotel,nothing,nice excel hotel,noth,nice excellent hotel,nothing,0.7579,0.0000,0.7579,0.0000
3,341,7.8,Novotel Amsterdam City,Austria,NaN,0,2501,Slightly out of the city center but very easy...,27,34,...,slightly city center easy get tram train reall...,NaN,slightli citi center easi get tram train reall...,NaN,slightly city center easy get tram train reall...,NaN,0.9414,0.0000,0.9414,0.0000
4,843,7.8,Hilton London Kensington Hotel,Qatar,NaN,0,3801,Great easy to reAch location,6,5,...,great easy reach location,NaN,great easi reach locat,NaN,great easy reach location,NaN,0.7964,0.0000,0.7964,0.0000


# Preperation of Paris Hotel Data

In [ ]:
# show list of columns
df.columns

In [ ]:
# show country column
df.country.unique()

In [ ]:
# Select Paris
df_paris = df[df.city == 'Paris']

In [ ]:
# View the data
df_paris.info()

In [ ]:
# View null values
df_paris.isnull().sum()


In [ ]:
df_paris.head()

## Explore Questions
Does a reviewer nationality affect the score given?

Does the total amount of reviews affect the review score?

Do the tags determine the reviewer score or nationality? (i.e. does certain words attract certain guests?)

What is the distribution of the customer's home country?

In [ ]:
# select average_score and review_nationality
df_score_nat = df_paris[['average_score','reviewer_nationality']]

In [ ]:
# the average score for each reviewer_national
nat_avg_score = df_score_nat.groupby('reviewer_nationality').mean()

In [ ]:
# how many unique reviewer_nationality there are
nat_avg_score

In [ ]:
df = nat_avg_score.reset_index()

In [ ]:
df.reviewer_nationality.unique()

In [ ]:
# create a list of reviewer_nationality
df.rename(columns={"reviewer_nationality": "Country"})

In [ ]:
# remove first column in df
df = df.drop(0)

In [ ]:
df

Does not look like there is any fluctuation in score based of nationality.

Does the length of a review determine if it is negative or positive?

In [ ]:
df_paris.head()

In [ ]:
df_paris.dtypes

In [ ]:
# view null values
df_paris.isnull().sum()

In [ ]:
df_paris.isnull().sum()

In [ ]:
df_paris.positive_review = df_paris.positive_review.fillna('no positive')

In [ ]:
df_paris.nights_stayed = df_paris.nights_stayed.fillna(df_paris.nights_stayed.mean())

In [ ]:
#drop lat and lng columns
df_paris = df_paris.drop(['lat','lng'], axis=1)

In [ ]:
df_paris.isnull().sum()

Data prep:
I filled NAN in neg_review with 'no negative'
I filled NAN in pos_review with 'no positive'
Filled NAN in nights stayed with average nights stayed
dropped lat and lng columns

In [ ]:
# imports
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats
import sklearn.preprocessing
from sklearn.model_selection import train_test_split

np.random.seed(246)

In [ ]:
# split the data
X_train_and_valid, X_test = train_test_split(df_paris, random_state=246)
X_train, X_valid = train_test_split(X_train_and_valid, random_state=246)

In [ ]:
X_train.shape

In [ ]:
# Select only the int and float columns
X_train_scale = X_train.select_dtypes(include=[np.int64, np.float64])

In [ ]:
# split the X_train_scale into descrete and continuous dataframes
X_train_scale_discrete = X_train_scale.select_dtypes(include=[np.int64])
X_train_scale_continuous = X_train_scale.select_dtypes(include=[np.float64])

In [ ]:
# MINMAX SCALER
scaler = sklearn.preprocessing.MinMaxScaler()
# fit scaler on training data
scaler.fit(X_train_scale_discrete)

In [ ]:
X_train_scaled = scaler.transform(X_train_scale_discrete)

In [ ]:
# convert X_train_scaled to dataframe
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train_scale_discrete.columns)

In [ ]:
X_train_scaled

In [ ]:
X_train_scale_continuous

In [ ]:
result = pd.concat([X_train_scale_continuous.reset_index(), X_train_scaled.reset_index()], axis=1)

In [ ]:
result.columns

In [ ]:
result.head()

In [ ]:
# look for correlations between variables
corr = result.corr()

In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(corr, annot=True, cmap='RdYlGn_r')
plt.show()

Take aways form heat map:

Target is reviewer score

Correlations:

- +0.37 average score

- +0.44 pos sentiment and pos lem sentiment

- -0.37 review total negative word count

- -0.38 negative unique word count

- +0.25 pos unique word count

In [ ]:
# drop columns
columns_to_keep = ['reviewer_score', 'average_score', 'pos_sentiment_score', 'pos_lem_sentiment_score', 'review_total_negative_word_counts', 'review_total_positive_word_counts', 'negative_unique_word_count', 'positive_unique_word_count']

# create a new df with columns to keep
X_train_scaled_new = result[columns_to_keep]

In [ ]:
X_train_scaled_new.head()

In [ ]:
target = X_train_scaled_new.reviewer_score

In [ ]:
# plot the distribution of the target variable
sns.relplot(x='average_score', y=target, data=X_train_scaled_new, kind='line')
plt.title('Average Score vs. Reviewer Score')
plt.show()
sns.relplot(x='pos_sentiment_score', y=target, data=X_train_scaled_new, kind='line')
plt.title('Positive Sentiment Score vs. Reviewer Score')
plt.show()
sns.relplot(x='pos_lem_sentiment_score', y=target, data=X_train_scaled_new, kind='line')
plt.title('Positive Lemma Sentiment Score vs. Reviewer Score')
plt.show()
sns.relplot(x='review_total_negative_word_counts', y=target, data=X_train_scaled_new, kind='line')
plt.title('Total Negative Word Count vs. Reviewer Score')
plt.show()
sns.relplot(x='review_total_positive_word_counts', y=target, data=X_train_scaled_new, kind='line')
plt.title('Total Positive Word Count vs. Reviewer Score')
plt.show()
sns.relplot(x='negative_unique_word_count', y=target, data=X_train_scaled_new, kind='line')
plt.title('Negative Unique Word Count vs. Reviewer Score')
plt.show()

Setiment analysis scores 

In [ ]:
df.head()

In [ ]:
words = ['nothing', 'n', 'none', 'nothing really', 'good', 'nothing dislike', 'liked everything', 'everything perfect', 'nil', 'nothing complain', 'nothing say']

# spell checker/Clustering

In [ ]:
from textblob import TextBlob 
from textblob import Word

from sklearn.feature_extraction.text import CountVectorizer
import nltk 
from nltk.corpus import stopwords

In [ ]:
df = wrangle.wrangle_hotel()

In [ ]:
df.head()

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns

from sklearn.cluster import KMeans

In [ ]:
# clustering
X = df[['review_total_positive_word_counts', 'pos_sentiment_score']]

In [ ]:
# Check the inertia of the KMeans model
with plt.style.context('seaborn-whitegrid'):
    plt.figure(figsize=(9, 6))
    pd.Series({k: KMeans(k).fit(X).inertia_ for k in range(2, 12)}).plot(marker='x')
    plt.xticks(range(2, 12))
    plt.xlabel('k')
    plt.ylabel('inertia')
    plt.title('Change in inertia as k increases')

In [ ]:
# KMeans with k=4
kmeans = KMeans(n_clusters=4)
kmeans.fit(X)

kmeans.predict(X)

In [ ]:
# Add a column to the iris dataframe called cluster
df['cluster1'] = kmeans.predict(X)

In [ ]:
# create three subplots with original data, actual species, and cluster assignments
with plt.style.context('seaborn-whitegrid'):
    plt.figure(figsize=(20, 9))
    plt.subplot(1, 2, 1)
    sns.scatterplot(x='review_total_positive_word_counts', y='pos_sentiment_score', data=df)
    plt.title('Original Data')

    plt.subplot(1, 2, 2)
    sns.scatterplot(x='review_total_positive_word_counts', y='pos_sentiment_score', hue='cluster1', data=df)
    plt.title('Original Data with Cluster Labels')

In [ ]:
def cluster(df, feature1, feature2, k):
    X = df[[feature1, feature2]]

    kmeans = KMeans(n_clusters=k).fit(X)
    
    df['cluster'] = kmeans.predict(X)

    centroids = pd.DataFrame(kmeans.cluster_centers_, columns=X.columns)
    
    plt.figure(figsize=(14, 9))
    
    for cluster, subset in df.groupby('cluster'):
        plt.scatter(subset[feature2], subset[feature1], label='cluster ' + str(cluster), alpha=.6)
    
    centroids.plot.scatter(y=feature1, x=feature2, c='black', marker='x', s=1000, ax=plt.gca(), label='centroid')
    
    plt.legend()
    plt.xlabel(feature1)
    plt.ylabel(feature2)
    plt.title('Visualizing Cluster Centers')

    return

In [ ]:
cluster(df, 'review_total_positive_word_counts', 'pos_sentiment_score', 4)

In [ ]:
cluster(df, 'reviewer_score', 'average_score', 4)

In [ ]:
cluster(df, 'total_number_of_reviews_reviewer_has_given', 'review_total_negative_word_counts', 3)

In [ ]:
cluster(df, 'total_number_of_reviews_reviewer_has_given', 'additional_number_of_scoring', 3)